# DS5110 Final Project, Dec 2025
By Brandon G. Villatla Lopez & Rachita Sharma

For the Maine Trust for Local News (METLN)

## Data Preparation Code

In this notebook we'll be reading the .xlxs files from the METLN_Data directory, doing some data cleaning, and saving it all into one table which we'll export as a .csv file

In [1]:
# import packages
import numpy as np
import pandas as pd

# we'll also need to have installed 'openpyxl' for the pd.read_excel() to work 
# it was included in the requirements!

### Read Raw Data

In [2]:
# I'll read one of the files to give an idea of what we have
# it might take a little longer than expected :')
test_df = pd.read_excel('METLN_Data/sublist2.1.24.xlsx')
test_df.head()

,Publication,AccoutID,Status,Bill Method,Dist ID,Route ID,Day pattern,City,State,Zip,Rate Code,LastStartDate,OriginalStartDate,OccupantID,RouteType ID
0,AMG_TR,2389901,Active,Office Pay,NaN,NaN,FriOnly,Ringwood,NJ,7456,TR_FriOnly_M_Mail,2020-06-26,2019-11-04,1917706.0,NaN
1,AMG_TR,2389906,Active,Office Pay,NaN,NaN,FriOnly,Austin,TX,78732,TR_FriOnly_M_Mail,2021-03-12,2019-11-04,1917711.0,NaN
2,AMG_TR,2395490,Active,Auto Pay - CC,NaN,NaN,FriOnly,Medford,NJ,8055,TR_FriOnly_M_Mail,2024-07-12,2019-12-06,1922411.0,NaN
3,AMG_TR,2432361,Active,Office Pay,NaN,NaN,FriOnly,Jasper,TN,37347,TR_FriOnly_M_Mail,2023-05-12,2021-04-23,1955588.0,NaN
4,AMG_TR,2460639,Active,Office Pay,NaN,NaN,FriOnly,Campbell Hall,NY,10916,TR_FriOnly_M_Mail,2024-05-03,2023-04-21,1980162.0,NaN


In [3]:
# Now that we know what we have, let's grab the data we need,
# clean it up, and save it in a better format

def fix_zip(zip_code:str): # we noticed zip codes were saved as ints and leading zeroes were missing
    '''
    This function takes in a zip code (str) and checks if it's less
    than 5-digits long. If it isn't, we assume it's alright and the
    same zip code is returned. If it's less, we assume leading zeros
    were dropped because of it being saved as an int and add them as
    necessary to make it 5-digits long. 
    '''
    zip_code = str(zip_code) # make sure it's a string!
    while len(zip_code)<5:
        zip_code = '0'+zip_code
    return zip_code
    
def read_file(month:str):
    '''
    This function will read in the specific file for a given month 
    (given as a string "2"-"10"). It will also prepare the dataset 
    for our use (e.g, creating "Month" and "Online" columns, fixing
    typos, and selecting columns of interest).
    '''
    month = str(month) # making sure it's a string, again!
    filename = 'METLN_Data/sublist'+month+'.1.24.xlsx'
    col_type = {'Publication' : str,
                'AccoutID' : str, # the column is mispelled
                'Status' : str,
                'Bill Method' : str,
                'Dist ID' : str,
                'Rout ID' : str,
                'Day pattern' : str,
                'City' : str,
                'State' : str,
                'Zip' : str,
                'Rate Code' : str,
                'LastStartDate' : 'datetime64[ns]',
                'OriginalStartDate' : 'datetime64[ns]',
                'OccupantID' : str,
                'RouteTypeID' : str}

    # read in the file
    df = pd.read_excel(filename, dtype=col_type)

    # fix data
    df['Zip'] = df.Zip.apply(fix_zip) # zip codes
    df = df.rename(columns={'AccoutID':'AccountID'}) # column typo

    # create month col and Online column
    df['Month'] = int(month)
    # remove na's from day pattern col (we don't know if online/physical)
    df = df[~df['Day pattern'].isna()]
    df['Online'] = df['Day pattern'] == 'O7Day'

    # keep only interesting col's
    columns = ['Publication', 'AccountID', 'Status', 'Month', 'Online', 'City', 'State', 'Zip', 'OccupantID']
    df = df[columns]
    return df

In [4]:
# we have data from Feb ('2') to Oct ('10'), let's read it and save it into one table
# THIS MIGHT TAKE A WHILE. xlxs files are slow to read :(
for month in range(2,11):
    if month == 2:
        df = read_file(month)
    else:
        df = pd.concat([df,read_file(month)], ignore_index = True)
df.head()

,Publication,AccountID,Status,Month,Online,City,State,Zip,OccupantID
0,AMG_TR,2389901,Active,2,False,Ringwood,NJ,07456,1917706
1,AMG_TR,2389906,Active,2,False,Austin,TX,78732,1917711
2,AMG_TR,2395490,Active,2,False,Medford,NJ,08055,1922411
3,AMG_TR,2432361,Active,2,False,Jasper,TN,37347,1955588
4,AMG_TR,2460639,Active,2,False,Campbell Hall,NY,10916,1980162


In [5]:
# just checking the type of each column
for col in df.columns:
    print(col,': ',df[col].dtype)

Publication :  object
AccountID :  object
Status :  object
Month :  int64
Online :  bool
City :  object
State :  object
Zip :  object
OccupantID :  object


### Clean-up text columns

In [6]:
print(df.State.unique()) # checking for spelling consistency

['NJ' 'TX' 'TN' 'NY' 'ME' 'MA' 'WV' 'OK' 'CT' 'HI' 'FL' 'NC' 'MD' 'NH'
 'PA' 'VA' 'MN' 'DC' 'IN' 'WA' 'CA' 'WI' 'VT' 'SD' 'SC' 'MT' 'OR' 'AK'
 'AZ' 'GA' 'CO' 'IL' 'IA' 'MI' 'RI' 'KS' 'AL' 'MS' 'KY' 'Me' 'ID' 'AR'
 'DE' 'OH' 'MO' 'NM' 'AA' 'MP' 'UT' 'LA' 'AP' 'ON' 'ND' 'NV' 'ma' 'Pa'
 'NE' 'me' 'nc' 'WY' 'AE' 'Ia' 'NL' 'PR']


In [7]:
# there is some inconsistency that'll hinder data aggregation (e.g., 'ME' vs 'me')
# so we fix it 
df['State'] = [state.upper() for state in df.State]
print(df.State.unique()) # should be fixed!

['NJ' 'TX' 'TN' 'NY' 'ME' 'MA' 'WV' 'OK' 'CT' 'HI' 'FL' 'NC' 'MD' 'NH'
 'PA' 'VA' 'MN' 'DC' 'IN' 'WA' 'CA' 'WI' 'VT' 'SD' 'SC' 'MT' 'OR' 'AK'
 'AZ' 'GA' 'CO' 'IL' 'IA' 'MI' 'RI' 'KS' 'AL' 'MS' 'KY' 'ID' 'AR' 'DE'
 'OH' 'MO' 'NM' 'AA' 'MP' 'UT' 'LA' 'AP' 'ON' 'ND' 'NV' 'NE' 'WY' 'AE'
 'NL' 'PR']


In [8]:
print(df.City.unique()[:30]) # checking the same - spoiler alert, it's worse

['Ringwood' 'Austin' 'Medford' 'Jasper' 'Campbell Hall' 'Brunswick'
 'Waterville' 'Bowdoinham' 'Bailey Island' 'Harpswell' 'Arrowsic'
 'Phippsburg' 'Topsham' 'Georgetown' 'Woolwich' 'WOOLWICH' 'Lisbon Falls'
 'Lisbon' 'Norfolk' 'Martinsburg' 'Choctaw' 'New Haven' 'Orrs Island'
 'Augusta' 'Bath' 'West Bath' 'South Portland' 'Freeport' 'Durham'
 'Pownal']


In [9]:
# no need to even look at it all, we can already see a few examples like 'Woolwich' and 'WOOLWICH'
# so we fix it, again :)
df['City'] = [str(city).title() for city in df.City]
print(df.City.unique()[:30])

['Ringwood' 'Austin' 'Medford' 'Jasper' 'Campbell Hall' 'Brunswick'
 'Waterville' 'Bowdoinham' 'Bailey Island' 'Harpswell' 'Arrowsic'
 'Phippsburg' 'Topsham' 'Georgetown' 'Woolwich' 'Lisbon Falls' 'Lisbon'
 'Norfolk' 'Martinsburg' 'Choctaw' 'New Haven' 'Orrs Island' 'Augusta'
 'Bath' 'West Bath' 'South Portland' 'Freeport' 'Durham' 'Pownal'
 'Cumberland']


### Save as .csv

In [10]:
# there are still other inconsistencies that could be fixed 
# (e.g., 'S Portland' vs 'South Portland'), but they're rare
# so we're stopping here and saving what we have 

df.to_csv('METLN_Data/clean_data.csv', index=False)